[실습] titanic3.csv 파일로 생존자 예측 모델 생성을 위한 분석용 데이터 셋을 작성하세요.

In [1]:
from google.colab import files
uploaded = files.upload()

Saving titanic3.csv to titanic3.csv


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [200]:
file_path = 'titanic3.csv'
df = pd.read_csv(file_path)

print(df)

      pclass  survived                                             name  \
0          1         1                    Allen, Miss. Elisabeth Walton   
1          1         1                   Allison, Master. Hudson Trevor   
2          1         0                     Allison, Miss. Helen Loraine   
3          1         0             Allison, Mr. Hudson Joshua Creighton   
4          1         0  Allison, Mrs. Hudson J C (Bessie Waldo Daniels)   
...      ...       ...                                              ...   
1304       3         0                             Zabour, Miss. Hileni   
1305       3         0                            Zabour, Miss. Thamine   
1306       3         0                        Zakarian, Mr. Mapriededer   
1307       3         0                              Zakarian, Mr. Ortin   
1308       3         0                               Zimmerman, Mr. Leo   

         sex    age  sibsp  parch  ticket      fare    cabin embarked boat  \
0     female  29.00  

In [201]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pclass     1309 non-null   int64  
 1   survived   1309 non-null   int64  
 2   name       1309 non-null   object 
 3   sex        1309 non-null   object 
 4   age        1046 non-null   float64
 5   sibsp      1309 non-null   int64  
 6   parch      1309 non-null   int64  
 7   ticket     1309 non-null   object 
 8   fare       1308 non-null   float64
 9   cabin      295 non-null    object 
 10  embarked   1307 non-null   object 
 11  boat       486 non-null    object 
 12  body       121 non-null    float64
 13  home.dest  745 non-null    object 
dtypes: float64(3), int64(4), object(7)
memory usage: 143.3+ KB


In [202]:
# 결측치 확인
print(df.isnull().sum())

pclass          0
survived        0
name            0
sex             0
age           263
sibsp           0
parch           0
ticket          0
fare            1
cabin        1014
embarked        2
boat          823
body         1188
home.dest     564
dtype: int64


In [203]:
# 필요없는 열 제거
df.drop(columns =['name', 'ticket', 'body', 'home.dest', 'boat', 'cabin', 'pclass'], inplace = True)
df.head()

,survived,sex,age,sibsp,parch,fare,embarked
0,1,female,29.00,0,0,211.3375,S
1,1,male,0.92,1,2,151.5500,S
2,0,female,2.00,1,2,151.5500,S
3,0,male,30.00,1,2,151.5500,S
4,0,female,25.00,1,2,151.5500,S


In [204]:
# 결측치 처리
df['age'].fillna(df['age'].mean(), inplace = True)
df['fare'].fillna(df['fare'].mean(), inplace = True)
df['embarked'].fillna(df['embarked'].mode()[0], inplace = True)
df
# df['pclass'].fillna(df['pclass'].mode()[0], inplace = True)

,survived,sex,age,sibsp,parch,fare,embarked
0,1,female,29.000000,0,0,211.3375,S
1,1,male,0.920000,1,2,151.5500,S
2,0,female,2.000000,1,2,151.5500,S
3,0,male,30.000000,1,2,151.5500,S
4,0,female,25.000000,1,2,151.5500,S
...,...,...,...,...,...,...,...
1304,0,female,14.500000,1,0,14.4542,C
1305,0,female,29.881138,1,0,14.4542,C
1306,0,male,26.500000,0,0,7.2250,C
1307,0,male,27.000000,0,0,7.2250,C


In [205]:
# age_group을 6개 범주로 확대
bins = [0, 5, 12, 18, 35, 60, 100]
labels = ['Infant', 'Child', 'Teenager', 'Young Adult', 'Adult', 'Senior']
df['age_group'] = pd.cut(df['age'], bins = bins, labels = labels)
df.drop(columns = ['age'], inplace = True)

In [206]:
# fare를 3개 범주로 변환
fare_bins = [0, 30, 100, 500]
fare_labels = ['Low', 'Medium', 'High']
df['fare_group'] = pd.cut(df['fare'], bins = fare_bins, labels = fare_labels)
df.drop(columns = ['fare'], inplace = True)

In [207]:
# sibsp와 parch 합쳐서 가족 규모 변수 생성
df['family_size'] = df['sibsp'] + df['parch'] # 자기 자신 포함해서 생성
df.drop(columns = ['sibsp', 'parch'], inplace = True)

In [208]:
# 범주형 변수를 더미 변수로 변환할 열 목록
categorical_columns = ['age_group', 'fare_group', 'sex', 'embarked']

# 반복문을 사용하여 범주형 변수를 더미 변수로 변환
for column in categorical_columns:
    # 첫 번째 카테고리는 drop_first = True 로 제거
    df = pd.get_dummies(df, columns = [column], drop_first = True)


In [209]:
df.head()

,survived,family_size,age_group_Child,age_group_Teenager,age_group_Young Adult,age_group_Adult,age_group_Senior,fare_group_Medium,fare_group_High,sex_male,embarked_Q,embarked_S
0,1,0,False,False,True,False,False,False,True,False,False,True
1,1,3,False,False,False,False,False,False,True,True,False,True
2,0,3,False,False,False,False,False,False,True,False,False,True
3,0,3,False,False,True,False,False,False,True,True,False,True
4,0,3,False,False,True,False,False,False,True,False,False,True


In [210]:
# 타겟 변수와 피처 변수 설정
X = df.drop('survived', axis=1)
y = df['survived']

# 학습 데이터와 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 크기 확인
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# 랜덤 포레스트 모델 학습
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

# 성능 평가
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


(1047, 11) (262, 11) (1047,) (262,)
Accuracy: 0.7633587786259542
Confusion Matrix:
 [[121  23]
 [ 39  79]]
Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.84      0.80       144
           1       0.77      0.67      0.72       118

    accuracy                           0.76       262
   macro avg       0.77      0.75      0.76       262
weighted avg       0.76      0.76      0.76       262



#### Task1. train.csv 파일로 생존자 예측 모델 생성을 위한 분석용 데이터 셋을 작성하세요.

### 1번째 방법
1. 'Age'는 기본적으로 NaN값을 NaN값이 아닌 사람들의 평균으로 채움.
2. 'Fare'가 0인 값들은 'Embarked'가 'S' 인것을 확인 함.
3. 2번 내용이 아닌 사람들의 평균을 0인 사람들로 채워서 계산함
4. 그 외에 가족 결합
5. 0.78

In [211]:
uploaded = files.upload()

Saving train.csv to train (1).csv


In [296]:
file_path = 'train.csv'

df = pd.read_csv(file_path)
df.head(20)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [297]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [298]:
# 결측치 확인
print(df.isnull().sum())

# PassengerId: 승객 고유 식별 번호
# Survived: 생존 여부 (0 = 사망, 1 = 생존)
# Pclass: 티켓 등급 (1 = 1등급, 2 = 2등급, 3 = 3등급)
# Name: 승객 이름
# Sex: 성별
# Age: 나이
# SibSp: 형제자매 및 배우자의 수
# Parch: 부모 및 자녀의 수
# Ticket: 티켓 번호
# Fare: 운임료
# Cabin: 객실 번호
#
# Embarked : 승선한 항구 이름
# 'S': Southampton (사우스앰프턴)
# 'C': Cherbourg (셰르부르)
# 'Q': Queenstown (퀸스타운)
#
# Age는 흠 .. 평균으로 하자..
# Cabin 열은 삭제하자
# Embarked 결측 치를 다른 값으로 채우자 .

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [299]:
# 필요없는 열 제거
df.drop(columns =['PassengerId', 'Name', 'Ticket', 'Cabin', 'Pclass'], inplace = True)
df.head()

,Survived,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,1,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,0,male,35.0,0,0,8.0500,S


In [300]:
df.loc[df['Age'].isna()]

,Survived,Sex,Age,SibSp,Parch,Fare,Embarked
5,0,male,NaN,0,0,8.4583,Q
17,1,male,NaN,0,0,13.0000,S
19,1,female,NaN,0,0,7.2250,C
26,0,male,NaN,0,0,7.2250,C
28,1,female,NaN,0,0,7.8792,Q
...,...,...,...,...,...,...,...
859,0,male,NaN,0,0,7.2292,C
863,0,female,NaN,8,2,69.5500,S
868,0,male,NaN,0,0,9.5000,S
878,0,male,NaN,0,0,7.8958,S


In [301]:
# 'Age'가 NaN이 아닌 행들의 평균 계산
Age_mean = round(df[df['Age'].notnull()]['Age'].mean(), 2)
print(Age_mean)

29.7


In [302]:
# 'Age'가 NaN인 값들 평균으로 채우기.
df.loc[df['Age'].isna(), 'Age'] = Age_mean

print(df.loc[df['Age'] == 29.7])

     Survived     Sex   Age  SibSp  Parch     Fare Embarked
5           0    male  29.7      0      0   8.4583        Q
17          1    male  29.7      0      0  13.0000        S
19          1  female  29.7      0      0   7.2250        C
26          0    male  29.7      0      0   7.2250        C
28          1  female  29.7      0      0   7.8792        Q
..        ...     ...   ...    ...    ...      ...      ...
859         0    male  29.7      0      0   7.2292        C
863         0  female  29.7      8      2  69.5500        S
868         0    male  29.7      0      0   9.5000        S
878         0    male  29.7      0      0   7.8958        S
888         0  female  29.7      1      2  23.4500        S

[177 rows x 7 columns]


In [303]:
# Embarked 결측치 채우기
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace = True)
df.head(10)

,Survived,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,1,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,0,male,35.0,0,0,8.0500,S
5,0,male,29.7,0,0,8.4583,Q
6,0,male,54.0,0,0,51.8625,S
7,0,male,2.0,3,1,21.0750,S
8,1,female,27.0,0,2,11.1333,S
9,1,female,14.0,1,0,30.0708,C


In [304]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Sex       891 non-null    object 
 2   Age       891 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Embarked  891 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 48.9+ KB


In [305]:
print(df.isnull().sum())

Survived    0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64


In [306]:
print(df['Age'].max()) # 80
print(df['Age'].min()) # 0.42

80.0
0.42


In [307]:
# age_group을 6개 범 주로 확대
bins = [0, 5, 12, 19, 35, 60, 80]
labels = ['Infant', 'Child', 'Teenager', 'Young Adult', 'Adult', 'Senior']
df['Age_group'] = pd.cut(df['Age'], bins = bins, labels = labels)
df.drop(columns = ['Age'], inplace = True)

In [308]:
print(df)

     Survived     Sex  SibSp  Parch     Fare Embarked    Age_group
0           0    male      1      0   7.2500        S  Young Adult
1           1  female      1      0  71.2833        C        Adult
2           1  female      0      0   7.9250        S  Young Adult
3           1  female      1      0  53.1000        S  Young Adult
4           0    male      0      0   8.0500        S  Young Adult
..        ...     ...    ...    ...      ...      ...          ...
886         0    male      0      0  13.0000        S  Young Adult
887         1  female      0      0  30.0000        S     Teenager
888         0  female      1      2  23.4500        S  Young Adult
889         1    male      0      0  30.0000        C  Young Adult
890         0    male      0      0   7.7500        Q  Young Adult

[891 rows x 7 columns]


In [309]:
# 'Fare' 컬럼의 값이 0인 행들을 필터링
fare_zero_df = df[df['Fare'] == 0]
print(fare_zero_df)

# 90% 이상 생존 X / 남자 / 승선한 항구들의 이름이 모두 S이다.

     Survived   Sex  SibSp  Parch  Fare Embarked    Age_group
179         0  male      0      0   0.0        S        Adult
263         0  male      0      0   0.0        S        Adult
271         1  male      0      0   0.0        S  Young Adult
277         0  male      0      0   0.0        S  Young Adult
302         0  male      0      0   0.0        S     Teenager
413         0  male      0      0   0.0        S  Young Adult
466         0  male      0      0   0.0        S  Young Adult
481         0  male      0      0   0.0        S  Young Adult
597         0  male      0      0   0.0        S        Adult
633         0  male      0      0   0.0        S  Young Adult
674         0  male      0      0   0.0        S  Young Adult
732         0  male      0      0   0.0        S  Young Adult
806         0  male      0      0   0.0        S        Adult
815         0  male      0      0   0.0        S  Young Adult
822         0  male      0      0   0.0        S        Adult


In [310]:
# 'Fare'가 0이 아닌 행들의 평균 계산
fare_mean = round(df[df['Fare'] != 0]['Fare'].mean(), 2)
print(fare_mean)

32.76


In [311]:
# 'Fare'가 0이고 'Embarked'가 'S'인 값들을 평균 값으로 채우기
df.loc[(df['Fare'] == 0) & (df['Embarked'] == 'S'), 'Fare'] = fare_mean

print(df)

     Survived     Sex  SibSp  Parch     Fare Embarked    Age_group
0           0    male      1      0   7.2500        S  Young Adult
1           1  female      1      0  71.2833        C        Adult
2           1  female      0      0   7.9250        S  Young Adult
3           1  female      1      0  53.1000        S  Young Adult
4           0    male      0      0   8.0500        S  Young Adult
..        ...     ...    ...    ...      ...      ...          ...
886         0    male      0      0  13.0000        S  Young Adult
887         1  female      0      0  30.0000        S     Teenager
888         0  female      1      2  23.4500        S  Young Adult
889         1    male      0      0  30.0000        C  Young Adult
890         0    male      0      0   7.7500        Q  Young Adult

[891 rows x 7 columns]


In [312]:
print(df['Fare'].max())
print(df['Fare'].min())

512.3292
4.0125


In [313]:
# Fare를 6개 범주로 변환
Fare_bins = [0, 10 ,30, 80, 300, 513]
Fare_labels = ['Very Low', 'Low', 'Medium', 'High', 'insane']
df['Fare_group'] = pd.cut(df['Fare'], bins = fare_bins, labels = fare_labels)
df.drop(columns = ['Fare'], inplace = True)

In [314]:
print(df.head(20))

    Survived     Sex  SibSp  Parch Embarked    Age_group Fare_group
0          0    male      1      0        S  Young Adult        Low
1          1  female      1      0        C        Adult     Medium
2          1  female      0      0        S  Young Adult        Low
3          1  female      1      0        S  Young Adult     Medium
4          0    male      0      0        S  Young Adult        Low
5          0    male      0      0        Q  Young Adult        Low
6          0    male      0      0        S        Adult     Medium
7          0    male      3      1        S       Infant        Low
8          1  female      0      2        S  Young Adult        Low
9          1  female      1      0        C     Teenager     Medium
10         1  female      1      1        S       Infant        Low
11         1  female      0      0        S        Adult        Low
12         0    male      0      0        S  Young Adult        Low
13         0    male      1      5        S     

In [315]:
# sibsp와 parch 합쳐서 가족 규모 변수 생성
df['family_size'] = df['SibSp'] + df['Parch'] # 자기 자신 포함해서 생성
df.drop(columns = ['SibSp', 'Parch'], inplace = True)

In [316]:
# 범주형 변수를 더미 변수로 변환할 열 목록
categorical_columns = ['Age_group', 'Fare_group', 'Sex', 'Embarked']

# 반복문을 사용하여 범주형 변수를 더미 변수로 변환
for column in categorical_columns:
    # 첫 번째 카테고리는 drop_first = True 로 제거
    df = pd.get_dummies(df, columns = [column], drop_first = True)


In [317]:
# 타겟 변수와 피처 변수 설정
X = df.drop('Survived', axis=1)
y = df['Survived']

# 학습 데이터와 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 크기 확인
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# 랜덤 포레스트 모델 학습
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

# 성능 평가
print("Accuracy:", accuracy_score(y_test, y_pred))
# print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
# print("Classification Report:\n", classification_report(y_test, y_pred))


(712, 11) (179, 11) (712,) (179,)
Accuracy: 0.7877094972067039


In [318]:
# 1차 테스트
# (712, 11) (179, 11) (712,) (179,)
# Accuracy: 0.7877094972067039

### 2번째 시도 방법
1. 1번째 시도 방법과 큰 틀은 벗어나지 않음.
2. 다만, 'Age'를 나눌 때 'Mr와 Miss'가 이름에 들어간 사람들의 평균을 Mr, Miss에 넣고, 나머지 NaN값인 사람들은 다른 사람들의 평균으로 대입하여 계산

In [433]:
file_path = 'train.csv'

df = pd.read_csv(file_path)
df.head(20)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [415]:
df.drop(columns =['PassengerId', 'Ticket', 'Cabin', 'Pclass'], inplace = True)

In [416]:
# 이름에서 'Mr.' 또는 'Miss.'을 포함한 행 선택
mr_miss_rows = df[df['Name'].str.contains('Mr.|Miss.', case=False, na=False)]

# 'Age'가 NaN이 아닌 행 선택
mr_miss_rows_with_age = mr_miss_rows[mr_miss_rows['Age'].notnull()]

# 'Age'의 평균 계산
mr_miss_age_mean = mr_miss_rows_with_age['Age'].mean()

print("Mr. 또는 Miss.의 평균 나이:", mr_miss_age_mean)

# NaN이 있는 행의 'Age'에 Mr. 또는 Miss.의 평균 나이 삽입
df.loc[(df['Age'].isna()) & (df['Name'].str.contains('Mr.|Miss.', case=False, na=False)), 'Age'] = mr_miss_age_mean


Mr. 또는 Miss.의 평균 나이: 30.625190839694657


In [417]:
# 'Age' NaN 값 있는지 확인
df.loc[df['Age'].isna()]

,Survived,Name,Sex,Age,SibSp,Parch,Fare,Embarked
65,1,"Moubarek, Master. Gerios",male,NaN,1,1,15.2458,C
159,0,"Sage, Master. Thomas Henry",male,NaN,8,2,69.5500,S
176,0,"Lefebre, Master. Henry Forbes",male,NaN,3,1,25.4667,S
709,1,"Moubarek, Master. Halim Gonios (""William George"")",male,NaN,1,1,15.2458,C
766,0,"Brewe, Dr. Arthur Jackson",male,NaN,0,0,39.6000,C


In [418]:
# 'Age'가 NaN이 아닌 행들의 평균 계산
Age_mean = round(df[df['Age'].notnull()]['Age'].mean(), 2)
print(Age_mean)

29.88


In [419]:
# 'Age'가 NaN인 값들 평균으로 채우기.
df.loc[df['Age'].isna(), 'Age'] = Age_mean

print(df.loc[df['Age'] == 29.88])

     Survived                                               Name   Sex    Age  \
65          1                           Moubarek, Master. Gerios  male  29.88   
159         0                         Sage, Master. Thomas Henry  male  29.88   
176         0                      Lefebre, Master. Henry Forbes  male  29.88   
709         1  Moubarek, Master. Halim Gonios ("William George")  male  29.88   
766         0                          Brewe, Dr. Arthur Jackson  male  29.88   

     SibSp  Parch     Fare Embarked  
65       1      1  15.2458        C  
159      8      2  69.5500        S  
176      3      1  25.4667        S  
709      1      1  15.2458        C  
766      0      0  39.6000        C  


In [420]:
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace = True)

In [421]:
bins = [0, 5, 12, 19, 35, 60, 80]
labels = ['Infant', 'Child', 'Teenager', 'Young Adult', 'Adult', 'Senior']
df['Age_group'] = pd.cut(df['Age'], bins = bins, labels = labels)
df.drop(columns = ['Age'], inplace = True)

In [422]:
fare_zero_df = df[df['Fare'] == 0]
print(fare_zero_df)

     Survived                              Name   Sex  SibSp  Parch  Fare  \
179         0               Leonard, Mr. Lionel  male      0      0   0.0   
263         0             Harrison, Mr. William  male      0      0   0.0   
271         1      Tornquist, Mr. William Henry  male      0      0   0.0   
277         0       Parkes, Mr. Francis "Frank"  male      0      0   0.0   
302         0   Johnson, Mr. William Cahoone Jr  male      0      0   0.0   
413         0    Cunningham, Mr. Alfred Fleming  male      0      0   0.0   
466         0             Campbell, Mr. William  male      0      0   0.0   
481         0  Frost, Mr. Anthony Wood "Archie"  male      0      0   0.0   
597         0               Johnson, Mr. Alfred  male      0      0   0.0   
633         0     Parr, Mr. William Henry Marsh  male      0      0   0.0   
674         0        Watson, Mr. Ennis Hastings  male      0      0   0.0   
732         0              Knight, Mr. Robert J  male      0      0   0.0   

In [423]:
fare_mean = round(df[df['Fare'] != 0]['Fare'].mean(), 2)
print(fare_mean)

32.76


In [424]:
df.loc[(df['Fare'] == 0) & (df['Embarked'] == 'S'), 'Fare'] = fare_mean

print(df)

     Survived                                               Name     Sex  \
0           0                            Braund, Mr. Owen Harris    male   
1           1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female   
2           1                             Heikkinen, Miss. Laina  female   
3           1       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female   
4           0                           Allen, Mr. William Henry    male   
..        ...                                                ...     ...   
886         0                              Montvila, Rev. Juozas    male   
887         1                       Graham, Miss. Margaret Edith  female   
888         0           Johnston, Miss. Catherine Helen "Carrie"  female   
889         1                              Behr, Mr. Karl Howell    male   
890         0                                Dooley, Mr. Patrick    male   

     SibSp  Parch     Fare Embarked    Age_group  
0        1      0   7.2500        S 

In [425]:
print(df['Fare'].max())
print(df['Fare'].min())

512.3292
4.0125


In [426]:
Fare_bins = [0, 10 ,30, 80, 300, 513]
Fare_labels = ['Very Low', 'Low', 'Medium', 'High', 'insane']
df['Fare_group'] = pd.cut(df['Fare'], bins = fare_bins, labels = fare_labels)
df.drop(columns = ['Fare'], inplace = True)

In [427]:
# sibsp와 parch 합쳐서 가족 규모 변수 생성
df['family_size'] = df['SibSp'] + df['Parch'] # 자기 자신 포함해서 생성
df.drop(columns = ['SibSp', 'Parch'], inplace = True)

# 범주형 변수를 더미 변수로 변환할 열 목록
categorical_columns = ['Age_group', 'Fare_group', 'Sex', 'Embarked']

In [428]:
# 반복문을 사용하여 범주형 변수를 더미 변수로 변환
for column in categorical_columns:
    # 첫 번째 카테고리는 drop_first = True 로 제거
    df = pd.get_dummies(df, columns = [column], drop_first = True)

In [429]:
df.drop(columns=['Name'], inplace=True)

In [431]:
# 타겟 변수와 피처 변수 설정
X = df.drop('Survived', axis=1)
y = df['Survived']

# 학습 데이터와 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 크기 확인
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# 랜덤 포레스트 모델 학습
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

# 성능 평가
print("Accuracy:", accuracy_score(y_test, y_pred))
# print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
# print("Classification Report:\n", classification_report(y_test, y_pred))

(712, 11) (179, 11) (712,) (179,)
Accuracy: 0.7877094972067039
